In [1]:
from params import log_offsets
from collections import deque

# LSTM imports  
import torch as T
import pandas as pd
from model import Model
from log_dataset_2 import LogDataset, TestDataset
from torch.utils.data import DataLoader 

type = 'euler'
cols = log_offsets[type]['cols']
buffer = deque()
pCols = []
for col in cols:
    c = "predicted_" + col
    pCols.append(c)
pdf = pd.DataFrame(columns=cols + pCols + ['label'])

base_file = './dataset/dji/baseline.csv'
test_file = 'anomaly_labeled.csv'

model_name = 'dji_' +  type
batch_size = 1
steps = 10  # data collection rate is set at 10Hz, hence 10 steps means prediction at every 1 second
window_sz = 30 # window size for runtime data analysis
threshold = 3 # deviation threshold 
epochs = 50 # for training
input_sz = len(cols)

djibaseDataset = LogDataset(base_file, cols, steps=steps)
djitestDataset = TestDataset(test_file, cols, steps=steps)
baseline = djibaseDataset

model = Model(batch_size=batch_size, input_size=3, hidden_size=64, num_layers=5, linear_size=128,device= 'cuda' if T.cuda.is_available() else 'cpu')
model.load_state_dict(T.load('models/' + model_name + '_s' + str(steps) + '_b' + str(batch_size)))
model.eval()  # set model in evaluation mode


Model(
  (lstm): LSTM(3, 64, num_layers=5, batch_first=True, dropout=0.2, bidirectional=True)
  (relu1): LeakyReLU(negative_slope=0.01)
  (fc_1): Linear(in_features=64, out_features=128, bias=True)
  (relu2): LeakyReLU(negative_slope=0.01)
  (fc_2): Linear(in_features=128, out_features=3, bias=True)
)

In [4]:
djiwindy_dl = DataLoader(djitestDataset, batch_size=100, shuffle=True, drop_last=True)
eval_iter = iter(djiwindy_dl)
next(eval_iter)[0]

ValueError: only one element tensors can be converted to Python scalars

In [12]:

def evaluate(dataloader, threshold_sigma=3, epochs = 10):
    std = T.tensor(baseline.df.describe().T['std'])
    std = T.round(std,decimals=10)
    std[std==0] = 0.0000000001 # must have a minimum std

    true_positives = 0.
    total = 0.
    # res = pd.DataFrame(columns=cols + pCols + ['label'])

    eval_iter = iter(dataloader)
    seq, seq_end, y = None, None

    # while True:
    #     try:
    seq, seq_end, y = next(eval_iter)
    
    seq_end_hat = model(seq)
    l1_distances = T.abs(seq_end_hat-seq_end)
    y_hat = 0
    for i,l1 in enumerate(l1_distances):
        total += 1
        threshold = threshold_sigma * std
        deviations = l1 > threshold
        deviation = T.any(deviations)
        
        if deviation.item() > 0: 
            label = 1
            true_positives += 1

In [13]:
djiwindy_dl = DataLoader(djitestDataset, batch_size=100, shuffle=True, drop_last=True)
evaluate(djiwindy_dl, threshold_sigma=threshold, epochs = 1)

ValueError: not enough values to unpack (expected 3, got 2)